In [1]:
import sys # required for relative imports in jupyter lab
sys.path.insert(0, '../') 

from scipy.sparse import coo_matrix
import numpy as np

from dataset import QM9

from torch_geometric.datasets import QM9 as TQM9
from torch_geometric.data import Data



RDKit WARNING: [11:19:34] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
ds_params = {'train_params': {'n': 1000,
                              'db': 'rdkit',
                              'n_conformers': 1,
                              'features': ['n_atoms','A','B','C','mu',
                                           'alpha','homo','lumo','gap','r2','zpve','Cv',
                                           'mulliken','distance','coulomb'],
                              'embeds': ['atom_types'],
                              'targets': ['U0'],
                              'pad': None,
                              'do_not_pad': ['U0','n_atoms','A','B','C','mu',
                                             'alpha','homo','lumo','gap','r2','zpve','Cv'],
                              #'filter_on': ('n_atoms','>','18'),
                              'use_pickle': 'qm9_test.p',
                              'flatten': True,
                              'embed_lookup': {'H': 0, 'C': 1, 'N': 2, 'O': 3, 'F': 4,
                                               'sp': 0, 'sp2': 1, 'sp3': 2, 'na': 3,
                                               0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6,
                                               7: 7, 8: 8, 9: 9, 10: 10, '0': 0}}}

qm9 = QM9(**ds_params['train_params'])

creating QM9 dataset...
molecules scanned:  1
molecules removed for no rdkit conformer:  0
molecules created:  1
molecules scanned:  1001
molecules removed for no rdkit conformer:  15
molecules created:  986
total molecules scanned:  1015
total molecules removed for no rdkit conformer:  15
total uncharacterized molecules removed:  26
total molecules created:  974
pickling a copy of the QM9 datadic...
CDataset created...


In [3]:
mol = qm9.ds[4]
mol

dsgdb9nsd_000004

In [4]:
qm9[4]

(array([ 4.0000000e+00,  0.0000000e+00,  3.5610035e+01,  3.5610035e+01,
         0.0000000e+00,  1.6280001e+01, -2.8450000e-01,  5.0600000e-02,
         3.3510000e-01,  5.9524799e+01,  2.6841000e-02,  8.5740004e+00,
        -2.0701900e-01, -2.0701900e-01,  2.0701900e-01,  2.0701900e-01,
         0.0000000e+00,  1.2198831e+00,  1.0639418e+00,  2.2811027e+00,
         1.2198831e+00,  0.0000000e+00,  2.2838078e+00,  1.0612266e+00,
         1.0639418e+00,  2.2838078e+00,  0.0000000e+00,  3.3450141e+00,
         2.2811027e+00,  1.0612266e+00,  3.3450141e+00,  0.0000000e+00,
         3.6858112e+01,  1.7582371e+01,  1.7815094e+01,  1.7858379e+01,
         1.7575157e+01,  3.6858112e+01,  1.7815094e+01,  1.7858379e+01,
         2.9291928e+00,  2.9303951e+00,  5.0000000e-01,  2.9763970e+00,
         2.9291928e+00,  2.9303951e+00,  2.9691825e+00,  5.0000000e-01],
       dtype=float32),
 [array([1, 1, 0, 0])],
 array([-77.308426], dtype=float32))

In [5]:
mol.coulomb

array([[36.858112 , 17.58237  , 17.815094 , 17.85838  ],
       [17.575157 , 36.858112 , 17.815094 , 17.85838  ],
       [ 2.9291928,  2.9303951,  0.5      ,  2.976397 ],
       [ 2.9291928,  2.9303951,  2.9691825,  0.5      ]], dtype=float32)

In [6]:
mol.smile

'C#C\tC#C\t\n'

In [7]:
mol.xyz

array([[-0.60771075,  0.58371377, -0.00612667],
       [ 0.60878495,  0.4932123 ,  0.00177013],
       [-1.66813387,  0.67010643, -0.00950269],
       [ 1.66705967,  0.41504267,  0.01385922]])

In [8]:
mol.qm9_xyz

array([[ 0.5995395,  0.       ,  1.       ],
       [-0.5995395,  0.       ,  1.       ],
       [-1.6616386,  0.       ,  1.       ],
       [ 1.6616386,  0.       ,  1.       ]], dtype=float32)

In [ ]:
mol.distance

In [ ]:
dir(mol)

In [ ]:
mol.rdmol.GetNumConformers()

In [ ]:
mol.rdmol

In [ ]:
confs = mol.rdmol.GetConformers()

In [ ]:
confs[0].GetPositions()